In [ ]:
import boto3
import pandas as pd
import os

In [ ]:
AWS_S3_BUCKET_NAME = 'border-crossing-bucket'
AWS_REGION = 'us-east-1'
AWS_ACCESS_KEY = os.environ.get('AWS_ACCESS_KEY')
AWS_SECRET_KEY = os.environ.get('AWS_SECRET_KEY')

In [ ]:
s3_client = boto3.client(
        service_name='s3',
        region_name=AWS_REGION,
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY
    )


In [ ]:
!wget -O border_crossings.csv https://data.transportation.gov/api/views/keg4-3bc2/rows.csv?accessType=DOWNLOAD
border_crossing_csv = 'border_crossings.csv'
bc_s3_file_name = 'border_crossings_s3.csv'
crossings_pre_processing = pd.read_csv(border_crossing_csv).drop(columns=['Point'])
crossings_pre_processing.to_csv(border_crossing_csv, index=False, header=False)
response = s3_client.upload_file(border_crossing_csv, AWS_S3_BUCKET_NAME, bc_s3_file_name)
print(response)

In [ ]:
!wget -O principal_ports.csv "https://hub.arcgis.com/api/v3/datasets/e3b6065cce144be8a13a59e03c4195fe_0/downloads/data?format=csv&spatialRefId=3857&where=1%3D1"
principal_ports_csv = 'principal_ports.csv'
pp_s3_file_name = 'principal_ports_s3.csv'
principal_ports_preproc = pd.read_csv(principal_ports_csv).drop(columns = ['X', 'Y'])
principal_ports_preproc.to_csv(principal_ports_csv, index=False, header=False)
response = s3_client.upload_file(principal_ports_csv, AWS_S3_BUCKET_NAME, pp_s3_file_name)
print(response)

In [ ]:
!wget -O port_statistical_areas.csv "https://hub.arcgis.com/api/v3/datasets/6755534edf0f441894e021912486db31_0/downloads/data?format=csv&spatialRefId=4269&where=1%3D1"
port_statistical_areas_csv = 'port_statistical_areas.csv'
ps_s3_file_name = 'port_statistical_areas_s3.csv'
port_statistical_areas_preproc = pd.read_csv(port_statistical_areas_csv)[['PORTIDPK', 'FEATUREDESCRIPTION']]
port_statistical_areas_preproc.to_csv(port_statistical_areas_csv, index=False, header=False)

response = s3_client.upload_file(port_statistical_areas_csv, AWS_S3_BUCKET_NAME, ps_s3_file_name)
print(response)

In [ ]:
import os
import urllib.request

import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

def process_file(url, file_name):
    with urllib.request.urlopen(url) as data:
        pd_data = pd.read_csv(data)
        if file_name == 'border_crossings':
            pd_data = pd_data.drop(columns=['Point'])
        elif file_name == 'principal_ports':
            pd_data = pd_data.drop(columns = ['X', 'Y'])
        elif file_name == 'port_statistical_areas':
            pd_data = pd_data[['PORTIDPK', 'FEATUREDESCRIPTION']]
        pa_table = pa.Table.from_pandas(pd_data)
        pq.write_table(pa_table, f'{file_name}_s3.parquet')

In [ ]:
url = "https://hub.arcgis.com/api/v3/datasets/6755534edf0f441894e021912486db31_0/downloads/data?format=csv&spatialRefId=4269&where=1%3D1"
process_file(url, 'port_statistical_areas')

In [ ]:
from airflow.providers.amazon.aws.hooks.s3 import S3Hook